In [31]:
import os

# csv文字型抽出
with open("201704_index_queries.csv", mode='rb') as f:
    binary = f.read()
text = chardet.detect(binary)
print(text)

{'encoding': 'CP932', 'confidence': 0.99, 'language': 'Japanese'}


In [32]:
import pandas as pd
import chardet
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

#カレントディレクトリ抽出
f = os.path.abspath(".")
files = os.listdir(f)

lists = []

#csvファイルリスト抽出
for i in files:
    if 'queries.csv' in i:
        lists.append(i)

In [103]:
# 集計用関数
def Aggregate(dfkw):
    csv = dfkw
    col = ['取得日','クエリ','インプレッション数','クリック数','CTR','平均掲載順位']
    s_query = "インプレッション数"
    rank = 9
    imp = 100

#     csv読み込み
    df_raw = pd.read_csv(csv ,encoding="CP932", index_col='通し番号')

    df_sort = df_raw.loc[:,col].sort_values(by=s_query, ascending=False)
    df_ck = df_sort[df_sort["クリック数"] >0]
    df_nck = df_sort[df_sort ["クリック数"] == 0]

    # クリック有、rank以上、imp以上
    df_h = df_ck[df_ck["平均掲載順位"] < rank
                ].sort_values(by=[s_query],ascending=False)
    
    df_high = df_h[df_h[s_query] > imp]

    # クリック有、rank以下、imp以上
    df_m = df_ck[df_ck["平均掲載順位"] > rank
                ].sort_values(by=[s_query],ascending=False)
    
    df_md = df_m[df_m[s_query] >imp]
    
    # クリック無、rank以上、imp以上
    df_l = df_nck[df_nck["平均掲載順位"]  < rank
                 ].sort_values(by=[s_query],ascending=False)
    
    df_low = df_l[df_l[s_query] > imp ]

    sortdata_list = [df_high, df_md, df_low,df_raw]
    return(sortdata_list)

In [111]:
"""
各条件でrawデータを加工しリストへappend
Dataframe型で使用したいのでpd.concatでリストをconcat
ここでは3つの条件で抽出したDataframeがdf_listsに格納されている
"""

# リスト作成
df_high = [] 
df_md = [] 
df_low = [] 
df_raw = []
df_lists = [df_high, df_md, df_low, df_raw]

# データセットリスト作成
for j, dfkw in enumerate(lists): #　lists = csvリスト
    df_list = Aggregate(dfkw)
    for m in range(len(df_list)):
        df_lists[m].append(df_list[m]) #簡略化

# データ統合
con_lists = []

for n in range(len(df_lists)):
    d = pd.concat(df_lists[n],  ignore_index=True)
    con_lists.append(d)

In [112]:
"""
ピポットにてクエリをキーにして年月でgrop_by
列のnull個数算出 + 判定列追加
注意点として直近浮上したデータはこの処理で排除される可能性があるためnull_numsには注意が必要
整形したデータを分析対象として使用する
"""

# ピポットデータの取得
ptdf_lists = []
value = ["インプレッション数","クリック数","CTR","平均掲載順位"]

for o in range(len(con_lists)):
    df_setpivot = con_lists[o].pivot_table(values = value
                                      , index = "クエリ", columns = "取得日"
                                      , aggfunc = 'sum'
#                                       , fill_value = 0
                                      )
    ptdf_lists.append(df_setpivot)


# level1リストの取得
df_col = ptdf_lists[0].columns.get_level_values(level=1)[1:8].unique() # dtype='object'

col_list = []
for r in range(len(df_col)):
    col_list.append(df_col[r])

# 変数定義
nlost_lists = []
query ="インプレッション数"
null_nums = 1

#  query列のnull個数算出 + 判定列追加
for p in  range(len(ptdf_lists)):
    null_num = 0
    
    # 検索query単体でファイルが存在する月の合計null個数カウント
    for s in range(len(col_list)):
            null_num += ptdf_lists[p][query][col_list[s]].isnull()*1

    # 検索query単体の月の合計null許容数
    df_set = ptdf_lists[p].assign(null_check = (null_num) > null_nums) 
    df_set = df_set.fillna(0)
    df_set = df_set.assign(mean = (df_set[query].mean(1)))
    df_set = df_set[df_set['null_check'] == False]
    nlost_lists.append(df_set)

In [117]:
"""
検索クエリベースでの評価用
nlost_lists
クリック有、rank以上、imp以上(num = 0)
クリック有、rank以下、imp以上(num = 1)
クリック無、rank以上、imp以上(num = 2)
"""
num = 0
q_query = 'インプレッション数'
condition_num = 500

df_sample = nlost_lists[num][q_query]
df_t = df_sample.T
df_sc = df_t.columns


"""
検索クエリベースでの評価
後ほど関数化
"""
# 検索クエリ一覧。文言抽出時に使用
sample_list = []
for u in df_sc:
    sample_list.append(u)

# # 単月最大q_queryがcondition_numを超えるクエリ
df_qover =  nlost_lists[num].assign(check = (df_t.max() > condition_num))
df_qover = df_qover[df_qover['check'] == True]

# # 単月最大q_queryがcondition_num以下のクエリ
df_qunder =  nlost_lists[num].assign(check = (df_t.max() > condition_num))
df_qunder = df_qunder[df_qunder['check'] == False]


In [114]:
# 単純書き出し用

"""
クリック有、rank以上、imp以上(m = 0)
クリック有、rank以下、imp以上(m = 1)
クリック無、rank以上、imp以上(m = 2)
"""
num = 2
out_cols = ["インプレッション数","クリック数","CTR","null_check"]
df_num = nlost_lists[num]

# dt_f = df_num[df_num['null_check']]
# dt_f = dt_f[out_cols]

In [109]:
# csv書き出し
# nlost_lists[3].to_csv(f + "/output/" + "df_raw.csv",
#               encoding="utf-8"
#              )

In [ ]:
# data_get_index  = dataset.index #index取得
# data_get_column_index = dataset.columns#カラム名index取得
# datalabel = dataset.loc[:,['クエリ','CTR']]#カラム名で絞り込み
# datatype = dataset.dtypes #データ型
# datasort = dataset.sort_values(by=["CTR"], ascending=False)#そーと
# dataset.loc[2:5,['CTR','クエリ']]
# type(dataset)
# d = d[['CTR']] > 0 #bool化
# dataset.columns == 'CTR'

#     slicename = csv[0:6] + csv[12:-4]

# data0_append = pd.concat(sortdata_lists[0],  ignore_index=True)#Notclick_NotImp_over0
# data1_append = pd.concat(sortdata_lists[1],  ignore_index=True)#Click_imp_over100
# data2_append = pd.concat(sortdata_lists[2],  ignore_index=True)#Click_imp_under100

#  インプレッション数null個数算出 + 列追加
# for p in  range(len(pivotdata_lists)):
#     data = pivotdata_lists[p].assign(
#         X = (pivotdata_lists[p]["インプレッション数"]['2017/06/04'].isnull()*1
#              + pivotdata_lists[p]["インプレッション数"]['2017/06/05'].isnull()*1
#              + pivotdata_lists[p]["インプレッション数"]['2017/07/06'].isnull()*1
#              + pivotdata_lists[p]["インプレッション数"]['2017/08/08'].isnull()*1
#              + pivotdata_lists[p]["インプレッション数"]['2017/09/07'].isnull()*1
#              + pivotdata_lists[p]["インプレッション数"]['2017/10/06'].isnull()*1
#              + pivotdata_lists[p]["インプレッション数"]['2017/11/05'].isnull()*1
#              + pivotdata_lists[p]["インプレッション数"]['2017/12/05'].isnull()*1
#             ) >=3 #nullの数
#     )
#     null_lost_lists.append(data)

# Series
# type(data0_pivot["CTR"]['2017/06/04'].isnull())


# matplot
# df_sample.plot(kind='bar', stacked=True)
# plt.show()